In [1]:
from tensorflow.keras.applications import DenseNet121

In [1]:
from keras.preprocessing.image import ImageDataGenerator
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data/train",target_size=(224,224),shuffle=True)


Found 2500 images belonging to 4 classes.


In [2]:
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from tensorflow.keras.optimizers import Adam



def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters


def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters


def dense_net(filters, growth_rate, classes, dense_block_size, layers_in_block):
    input_img = Input(shape=(224,224, 3))
    x = Conv2D(24, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = GlobalAveragePooling2D()(dense_x)

    output = Dense(classes, activation='softmax')(dense_x)

    return Model(input_img, output)

In [3]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="accuracy")
dense_block_size = 3
layers_in_block = 4

growth_rate = 12
classes = 4
model = dense_net(growth_rate * 2, growth_rate, classes, dense_block_size, layers_in_block)
model.summary()

# training
#batch_size = 32
epochs = 10
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
history=model.fit_generator(generator=traindata, epochs=epochs, shuffle=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 24  648         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 224, 224, 24  0           ['conv2d[0][0]']                 
                                )                                                             

C:\Users\fatim\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\fatim\AppData\Local\Temp/ipykernel_4980/2559335715.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=traindata, epochs=epochs, shuffle=True)


Epoch 1/10
79/79 [==============================] - 805s 10s/step - loss: 0.5978 - accuracy: 0.5320
Epoch 2/10
79/79 [==============================] - 778s 10s/step - loss: 0.4984 - accuracy: 0.5360
Epoch 3/10
79/79 [==============================] - 789s 10s/step - loss: 0.4738 - accuracy: 0.5308
Epoch 4/10
79/79 [==============================] - 796s 10s/step - loss: 0.4626 - accuracy: 0.5348
Epoch 5/10
79/79 [==============================] - 834s 11s/step - loss: 0.4558 - accuracy: 0.5376
Epoch 6/10
79/79 [==============================] - 773s 10s/step - loss: 0.4499 - accuracy: 0.5460
Epoch 7/10
79/79 [==============================] - 785s 10s/step - loss: 0.4461 - accuracy: 0.5532
Epoch 8/10
79/79 [==============================] - 779s 10s/step - loss: 0.4416 - accuracy: 0.5564
Epoch 9/10
79/79 [==============================] - 773s 10s/step - loss: 0.4399 - accuracy: 0.5628
Epoch 10/10
79/79 [==============================] - 52270s 670s/step - loss: 0.4353 - accuracy: 0.5

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
%matplotlib inline


In [38]:
from keras.preprocessing import image
img = image.load_img("notB.png",target_size=(224,224))
img = np.asarray(img)
img = np.expand_dims(img, axis=0)
test=model.predict(img)

In [5]:
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="data/test", target_size=(224,224))

Found 592 images belonging to 1 classes.


In [6]:
predictions = model.predict_generator(testdata)

C:\Users\fatim\AppData\Local\Temp/ipykernel_4980/3422549023.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(testdata)


In [9]:
pred_labels = list(np.argmax(predictions, axis=-1))

In [10]:
pred_labels

[2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,


In [86]:
list(traindata.class_indices.keys())[pred_labels[0]]

'Trans-thalamic'

In [11]:
df_test = pd.read_csv("test (1).csv")

In [12]:
true_labels_df=pd.read_csv("FETAL_PLANES_DB_data.csv",sep=';')

In [132]:
list(true_labels_df[true_labels_df['Image_name']=='Patient01563_Plane3_1_of_3']['Brain_plane'])[0]

'Trans-thalamic'

In [13]:

#true_labels
true_labels=[]
for im in df_test['image_name']:
    true_labels.append(list(true_labels_df[true_labels_df['Image_name']==im]['Brain_plane'])[0])
    

In [134]:
true_labels

['Trans-thalamic',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-cerebellum',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-ventricular',
 'Trans-cerebellum',
 'Trans-ventricular',
 'Trans-thalamic',
 'Trans-ventricular',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Other',
 'Other',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-ventricular',
 'Trans-cerebellum',
 'Other',
 'Trans-thalamic',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-ventricular',
 'Trans-ventricular',
 'Trans-cerebellum',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-ventricular',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-thalamic',
 'Trans-cerebellum',
 'Trans-thalamic'

In [14]:
im_names=df_test['image_name']

In [70]:
im_names[0]

'Patient01563_Plane3_1_of_3'

In [15]:
import csv

with open('sub2.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(['image_name',"class"])
    for i in range(len(im_names)):
        wr.writerow([im_names[i],list(traindata.class_indices.keys())[pred_labels[i]]])
    

In [16]:
d = pd.read_csv("sub2.csv")
labels=d['class']

In [136]:
labels

0        Trans-thalamic
1      Trans-cerebellum
2        Trans-thalamic
3        Trans-thalamic
4        Trans-thalamic
             ...       
587    Trans-cerebellum
588      Trans-thalamic
589    Trans-cerebellum
590    Trans-cerebellum
591    Trans-cerebellum
Name: class, Length: 592, dtype: object

In [17]:
from sklearn.metrics import f1_score
f1_score(true_labels, labels,average='micro')

0.43243243243243246